In [137]:
from faker import Faker
import numpy as np
import pandas as pd
import random as rd
from datetime import date

In [138]:
products_labels = ["id", "name", "price", "breeds", "is_for_dog", "is_for_cat"]
locals_vets_labels = ["id", "name", "has_fb", "has_ig", "lat", "lng","is_vet"]
sales_local_labels = ["id", "id_local", "id_client",
"name_client", "product_id", "product_name", "quantity",
"price", "total_price", "sale_by_publisher", "sale_online","date"]
# service_sales_local_labels = ["id", "id_local_vet", "id_client",
# "name_client", "id_service", "service_name",
# "price", "sale_by_publisher", "sale_online"]
services_vet_labels = ["id", "id_local_vet",
"name_pet", "breed", "price", "date",'is_online','is_cat','is_dog','service_name']


In [139]:
Faker.seed(0)
faker= Faker(['en-US'])
rd.seed(0)
sample_num=50

In [140]:
df_dog_breed=pd.read_csv('./dog_breed.csv')
df_cat_breed=pd.read_csv('./cats.csv')

In [141]:
dog_breeds=df_dog_breed.drop_duplicates(subset='breed').sample(sample_num).breed.to_list()
cat_breeds=df_cat_breed.drop_duplicates(subset='breed').sample(sample_num).breed.to_list()

In [142]:
pet_food=pd.read_csv('./openfoodfacts_export (1).csv',sep='\t')
pet_food=pet_food[["code","product_name_en","categories_tags"]]
pet_food.dropna(inplace=True)
pet_food["is_dog_cat"]=pet_food.categories_tags.apply(lambda x: 1 if "dog" in x else (2 if 'cat' in x else rd.randint(1,2)) )

In [143]:
d={
    'service_name':['Peluquería','Cortar uñas','Operación de pata','Baño médico','Desparasitación','Vacunación','servicio general','castración'],
    'prices':[30,20,1000,40,30,50,20,80]
}
df_services=pd.DataFrame(d)

In [144]:
def generate_products(quantity,df_products,dog_breeds,cat_breeds,range_price,products_labels):
    list_products=[]
    for i in range(quantity):
        sample=df_products.sample()
        code=int(sample['code'])
        name=sample['product_name_en'].values[0]
        is_dog_cat=sample['is_dog_cat'].values[0]
        is_for_cat=0
        is_for_dog=0
        breed=''
        if is_dog_cat==1:
            is_for_dog=1
            breed=rd.choice(dog_breeds)
        else:
            is_for_cat=1
            breed=rd.choice(cat_breeds)
        price=rd.random()*rd.randint(*range_price)
        list_products.append((code,name,price,breed,is_for_cat,is_for_dog))
    return pd.DataFrame(list_products,columns=products_labels)
        


In [145]:
def generate_locals(quantity,local_labels):
    list_locals=[]
    for i in range(quantity):
        id_local=i
        name_local=faker.company()
        has_ig,has_fb=faker.random_choices(elements=(1,0),length=2)
        geo_local=faker.local_latlng(country_code="EC")
        lat=geo_local[0]
        lng=geo_local[1]
        is_vet=rd.randint(0,1)
        list_locals.append((id_local,name_local,has_fb,has_ig,lat,lng,is_vet))
    return pd.DataFrame(list_locals, columns=local_labels)

In [146]:
def generate_sales_locals(quantity,df_locals,df_products,label_sales,range_date):
    list_sales=[]
    for i in range(quantity):
        index_local=rd.randint(0,df_locals.shape[0]-1)
        index_products=rd.randint(0,df_products.shape[0]-1)
        local=df_locals.iloc[index_local]
        product=df_products.iloc[index_products]
        id_local=local.id
        id_client=faker.ssn()
        name_cliente=faker.name()
        product_id=product.id
        product_name=product.name
        quantity_value=rd.randint(1,10)
        price=product.price
        total_price=price*quantity_value
        sale_by_publisher=rd.randint(0,1)
        prob=1
        suma_social_media=local.has_fb+ local.has_ig
        if suma_social_media==2:
            prob=0.6
        elif suma_social_media==1:
            prob=0.3
        else:
            prob=0
        
        sale_online= rd.random()<=prob
        start_date=date.fromisoformat(range_date[0])
        end_date=date.fromisoformat(range_date[1])
        date_f=faker.date_between(start_date,end_date)
        list_sales.append((i,id_local,id_client,
        name_cliente,product_id,product_name,quantity_value,
        price,total_price,sale_by_publisher,sale_online,date_f))

    return pd.DataFrame(list_sales,columns=label_sales)

In [147]:
def generate_services(quantity,df_locals,dogs_breed,cats_breed,range_date,services_labels,df_services):
    list_services=[]
    for i in range(quantity):
        index_local=rd.randint(0,df_locals.shape[0]-1)
        index_service=rd.randint(0,df_services.shape[0]-1)
        service=df_services.iloc[index_service]
        local=df_locals.iloc[index_local]
        suma_social_media=local.has_fb+ local.has_ig
        if suma_social_media==2:
            prob=0.9
        elif suma_social_media==1:
            prob=0.75
        else:
            prob=0
        is_online= rd.random()<=prob
        id_local_vet=local.id
        name_pet=faker.name_nonbinary()
        is_cat=rd.randint(0,1)
        is_dog=int(not is_cat)
        breed=''
        if is_cat:
            breed=rd.choice(cats_breed)
        else:
            breed=rd.choice(dogs_breed)
        price=service.prices
        start_date=date.fromisoformat(range_date[0])
        end_date=date.fromisoformat(range_date[1])
        date_f=faker.date_between(start_date,end_date)
        service_name=service.service_name

        list_services.append((i,id_local_vet,name_pet,breed,price,date_f,is_online,is_cat,is_dog,service_name))
    return pd.DataFrame(list_services,columns=services_labels)
        

In [148]:
range_price=(10,100)
range_date=('2021-05-01','2021-07-01')

In [149]:
df_product_gen=generate_products(200,pet_food,dog_breeds,cat_breeds,range_price,products_labels)
df_local_gen=generate_locals(100,locals_vets_labels)
df_sales_local_gen=generate_sales_locals(10000,df_local_gen,df_product_gen,sales_local_labels,range_date)
df_services_gen=generate_services(10000,df_local_gen[df_local_gen.is_vet==1],dog_breeds,cat_breeds,range_date,services_vet_labels,df_services)


In [150]:
root_path='./data'

In [151]:
df_product_gen.to_csv(f'{root_path}/products.csv',index=False)
df_local_gen.to_csv(f'{root_path}/locals.csv',index=False)
df_sales_local_gen.to_csv(f'{root_path}/local_sales.csv',index=False)
df_services_gen.to_csv(f'{root_path}/sale_services_vet.csv',index=False)